In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = False

import numpy as np
import seaborn as sns

import re
import folium


In [2]:
# importação de dados Brutos

csvList = ['cafeteria', 'delivery de comida', 'drive thru', 'padaria', 'restaurante']
dataFrames = []

for nome in csvList:
    # Lê o arquivo Bruto
    df = pd.read_csv(f"database/dadosBrutos/{nome}.csv", sep=';')

    # Dropa os registros duplicados em NOME e LOCAL deixando somente a primeira ocorrencia
    df = df.drop_duplicates(subset=['NOME', 'LOCAL'], keep='first')

    # Concatena os dados
    dataFrames.append(df)

# Cria um unico dataFrame Bruto
df_Bruto = pd.concat(dataFrames, ignore_index=True)

df_Bruto.to_csv("database/dadosTratados/bkp_df_Bruto.csv")


#df_Bruto


In [3]:

# Busco por ocorrencias de estabelecimentos que não são de alimentação
palavras_chave = [
    'restaurante', 'pizzaria', 'hamburgueria', 'cafeteria', 'doceria', 'confeitaria',
    'bar', 'delivery', 'sorveteria', 'padaria', 'comida', 'buffet', 'açaí', 'café',
    'carne', 'italiano', 'pizza', 'food', 'chá'
]

# Cria um filtro para os que são drive thru e que batem com as palavras-chave em TIPO
filtro_drive_thru = (
    (df_Bruto['BUSCA_ID'] == 'drive thru') &
    (df_Bruto['TIPO'].fillna('').str.lower().apply(lambda x: any(palavra in x for palavra in palavras_chave)))
)

# Cria um filtro para os que NÃO são drive thru (mantém tudo)
filtro_outras = (df_Bruto['BUSCA_ID'] != 'drive thru')

# Combina os dois: mantém todas as outras linhas + só os drive thru filtrados
filtro_final = filtro_outras | filtro_drive_thru

df_filtrado = df_Bruto[filtro_final]
df_filtrado.to_csv("database/dadosTratados/bkp_df_filtrado.csv")


#df_filtrado


In [4]:

# Tratamento da quantidade de comentarios
df_filtrado = df_filtrado.copy()
df_filtrado['NUM_COMENTARIO'] = df_filtrado['COMENTARIO'].str.replace('comentários', '', regex=False).str.replace(' ', '').str.replace('.', '')
df_filtrado['NUM_COMENTARIO'] = df_filtrado['NUM_COMENTARIO'].str.replace('Umcomentário', '1', regex=False)
df_filtrado['NUM_COMENTARIO'] = df_filtrado['NUM_COMENTARIO'].fillna(0)

# Tratamento da Pontuação, zerando registro nulos
df_filtrado['PONTUACAO'] = df_filtrado['PONTUACAO'].fillna(0)

# Remove a coluna COMENTARIO deixando somente a NUM_COMENTARIO
df_filtrado = df_filtrado.drop(columns='COMENTARIO')

#df_filtrado


In [5]:

# Tratar agora o tempo de espera

# Função para extrair o tempo da string
def extrair_hora(texto):
    try:
        if not texto or not isinstance(texto, str):
            return pd.Series([0, 0])
        
        texto = texto.replace(',', '.').strip().lower()
        partes = texto.split('a') if 'a' in texto else [texto]
        
        tempos = []

        for parte in partes:
            parte = parte.strip()

            match_min = re.search(r'(\d+\.?\d*)\s*min', parte)
            match_h = re.search(r'(\d+\.?\d*)\s*h', parte)

            if match_min:
                minutos = float(match_min.group(1))
                tempos.append(minutos)
            elif match_h:
                horas = float(match_h.group(1))
                tempos.append(horas * 60)
        
        if len(tempos) == 1:
            return pd.Series([tempos[0], tempos[0]])
        elif len(tempos) == 2:
            return pd.Series([min(tempos), max(tempos)])
        else:
            return pd.Series([0, 0])
    except:
        return pd.Series([0, 0])

df_filtrado[['TEMPO_MIN_M', 'TEMPO_MAX_M']] = df_filtrado['TEMPO_ESPERA'].apply(extrair_hora)

df_filtrado['TEMPO_MIN'] = pd.to_timedelta(df_filtrado['TEMPO_MIN_M'], unit='m')
df_filtrado['TEMPO_MAX'] = pd.to_timedelta(df_filtrado['TEMPO_MAX_M'], unit='m')

#df_filtrado


In [6]:
df_filtrado = df_filtrado.copy()

# Alteração dos nomes de coluna para melhor entendimento

# LOCAL PARA ENDERECO
df_filtrado = df_filtrado.rename(columns={'LOCAL': 'ENDERECO'})

# TIPO para LOCAL
df_filtrado = df_filtrado.rename(columns={'TIPO': 'LOCAL'})

# BUSCA_ID para TIPO
df_filtrado = df_filtrado.rename(columns={'BUSCA_ID': 'TIPO'})

# Mudando a ordenação da colunas somente para melhorar visualização

df_filtrado = df_filtrado[['NOME','TIPO','ENDERECO','LOCAL','PONTUACAO','NUM_COMENTARIO','TEMPO_MIN_M','TEMPO_MAX_M','TEMPO_MIN','TEMPO_MAX']]

df_final = df_filtrado.reset_index(drop=True)


#df_final



In [7]:

# Criar coluna BAIRRO
df_final['BAIRRO'] = df_final['ENDERECO'].str.split(', Fortaleza').str[0].str.split('-').str[-1].str.strip().str.title()

#df_final


In [8]:
# Salvando em um arquivo

df_final.to_csv("database/dadosTratados/df_final_semGeo.csv")
